# GiNZA
* 形態素解析
* 係り受け関係解析

In [15]:
import spacy
nlp = spacy.load("ja_ginza")

In [16]:
# パイプライン構成
for p in nlp.pipeline:
    print(p)

('tagger', <spacy.pipeline.pipes.Tagger object at 0x000002CD95E590D0>)
('parser', <spacy.pipeline.pipes.DependencyParser object at 0x000002CD961D7D60>)
('ner', <spacy.pipeline.pipes.EntityRecognizer object at 0x000002CD961D7D00>)
('CompoundSplitter', <ginza.compound_splitter.CompoundSplitter object at 0x000002CD9256E280>)
('BunsetuRecognizer', <ginza.bunsetu_recognizer.BunsetuRecognizer object at 0x000002CD92369B20>)


In [37]:
doc = nlp("ひき肉に、炒めて冷ましたたまねぎ、パン粉と牛乳と卵を混ぜる。")

In [38]:
# 形態素解析結果
for sent in doc.sents:
    for token in sent:
        print(token.i, token.orth_, token.lemma_, token.pos_, token.dep_, token.head.i)

0 ひき肉 ひき肉 NOUN obl 15
1 に に ADP case 0
2 、 、 PUNCT punct 0
3 炒め 炒める VERB advcl 5
4 て て SCONJ mark 3
5 冷まし 冷ます VERB acl 7
6 た た AUX aux 5
7 たまねぎ たまねぎ NOUN obl 15
8 、 、 PUNCT punct 7
9 パン粉 パン粉 NOUN nmod 11
10 と と ADP case 9
11 牛乳 牛乳 NOUN nmod 13
12 と と ADP case 11
13 卵 卵 NOUN obj 15
14 を を ADP case 13
15 混ぜる 混ぜる VERB ROOT 15
16 。 。 PUNCT punct 15


In [39]:
# Token
print('token-----------------------')
for token in doc:
  print(token.text)

token-----------------------
ひき肉
に
、
炒め
て
冷まし
た
たまねぎ
、
パン粉
と
牛乳
と
卵
を
混ぜる
。


In [40]:
# 名詞句のみ抽出
print('noun-----------------------')
for np in doc.noun_chunks:
  print(np)

noun-----------------------
ひき肉
炒めて冷ましたたまねぎ
パン粉
牛乳
卵


In [41]:
# 固有表現
print('ner-----------------------')
for ent in doc.ents:
  print(ent.text, ent.start_char, ent.end_char, ent.label_)


ner-----------------------
たまねぎ 12 16 Food_Other
パン粉 17 20 Food_Other
牛乳 21 23 Food_Other
卵 24 25 Food_Other


In [42]:
print(help(doc))

Help on Doc object:

class Doc(builtins.object)
 |  A sequence of Token objects. Access sentences and named entities, export
 |  annotations to numpy arrays, losslessly serialize to compressed binary
 |  strings. The `Doc` object holds an array of `TokenC` structs. The
 |  Python-level `Token` and `Span` objects are views of this array, i.e.
 |  they don't own the data themselves.
 |  
 |  EXAMPLE:
 |      Construction 1
 |      >>> doc = nlp(u'Some text')
 |  
 |      Construction 2
 |      >>> from spacy.tokens import Doc
 |      >>> doc = Doc(nlp.vocab, words=[u'hello', u'world', u'!'],
 |      >>>           spaces=[True, False, False])
 |  
 |  DOCS: https://spacy.io/api/doc
 |  
 |  Methods defined here:
 |  
 |  __bytes__(...)
 |  
 |  __getitem__(...)
 |      Get a `Token` or `Span` object.
 |      
 |      i (int or tuple) The index of the token, or the slice of the document
 |          to get.
 |      RETURNS (Token or Span): The token at `doc[i]]`, or the span at
 |          

抽出できる固有表現の種類ですが、上記の結果から推察するに、こちら*6だと思います。とは言っても GiNZA がどのようなデータで学習したか確認できてないので、はっきりしたことは言えません。ただ、どのような固有表現を抽出したいかは、要件次第のところもあります。GiNZA の git リポジトリに含まれる学習スクリプト*7を見ると、こちら*8 の train_data の形式で学習データを食わせてやれば良さそうなので、興味のある人は頑張ってみてください。spaCy のドキュメント*9によれば少なくとも数百件は必要とのことです。

In [43]:
# 係り受け関係表示
from nltk import Tree

# tree表示するトークンのフォーマット
def token_format(tk):
    # トークンのorth_、dep_、pos_の３つをトークンの情報として含める
    return "_".join([tk.orth_, tk.dep_, tk.pos_])

# tree表示する関数
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(token_format(node), [to_nltk_tree(child) for child in node.children])
    else:
        return token_format(node)

[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

                                                                  混ぜる_ROOT_VERB                                                                            
       _________________________________________________________________|_________________________________________________                                  
      |                       |                                   たまねぎ_obl_NOUN                                       卵_obj_NOUN                           
      |                       |                            _____________|____________                           __________|___________                      
      |                       |                           |                     冷まし_acl_VERB                   |                 牛乳_nmod_NOUN              
      |                       |                           |              ____________|_____________            |           ___________|_____________        
      |                  ひき肉_obl_NOUN                     |  

[None]

In [44]:
# 係り受け関係解析：グラフ表示
from spacy import displacy

for sent in doc.sents:
    svg = displacy.render(sent, style="dep", options={"compact":True})

In [45]:
# 係り受け関係解析：料理手順

nlp = spacy.load('ja_ginza')

instruction_list = [
  "ひき肉に、炒めて冷めたたまねぎ、パン粉と牛乳と卵を混ぜる。塩、胡椒、カレー粉を加えて、さらによく練る。小判上に成形して裏表にパン粉をつける。真ん中をくぼます。",
  "［ポーチドエッグ］水、酢、塩を入れて沸騰させ、割った卵を鍋の縁からゆっくり入れる。卵が茹で上がったら水に取り出す。",
  "赤ワインを火に掛けアルコールを飛ばし、水、トマトケチャップ、トマトピューレを合わせ、ソースを作る。",
  "熱したフライパンにサラダ油を入れ、中火でハンバーグのパテを入れる。1分程度焼く、裏返し、裏も1分程度焼く。ハンバーグを取り出し、フライパンの油を捨てる。",
  "中火にし、玉葱の輪切りを入れる。玉葱が焼けたら裏返し、ハンバーグを上に置く。のせたハンバーグにソースの材料を加え、フライパンにフタをし2～3分蒸し焼きにする。",
  "竹串で突き透明の肉汁が出たら取り出す。肉汁を煮立て、塩胡椒を加えソースを完成させる。ハンバーグを皿に盛り、玉葱の輪切りとポーチドエッグをのせ、ソースをかける。"
]

# instruction = ''.join(instruction_list)
docs = list(nlp.pipe(instruction_list))

for doc in docs:
  for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

# for sent in doc.sents:
#     svg = displacy.render(sent, style="dep")

たまねぎ 11 15 Food_Other
パン粉 16 19 Food_Other
牛乳 20 22 Food_Other
卵 23 24 Food_Other
塩 29 30 Food_Other
胡椒 31 33 Food_Other
カレー粉 34 38 Dish
パン粉 62 65 Food_Other
ポーチドエッグ 1 8 Doctrine_Method_Other
酢 11 12 Food_Other
塩 13 14 Food_Other
卵 26 27 Food_Other
卵 41 42 Food_Other
赤ワイン 0 4 Dish
ソース 42 45 Food_Other
ハンバーグ 20 25 Dish
1分程度 33 37 Period_Time
1分程度 46 50 Period_Time
ハンバーグ 53 58 Dish
油 70 71 Food_Other
玉葱 5 7 Food_Other
玉葱 16 18 Flora_Part
ハンバーグ 27 32 Dish
ハンバーグ 41 46 Dish
2～3分 67 71 Period_Time
肉汁 8 10 Animal_Part
肉汁 19 21 Animal_Part
ソース 32 35 Food_Other
ハンバーグ 42 47 Dish
玉葱 53 55 Food_Other
ソース 71 74 Food_Other
